In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_friedman1
from sklearn.metrics import mean_squared_error
import sklearn.model_selection as ms

In [2]:
import numpy as np
np.random.seed(123)
import collections, copy, pickle
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [3]:
from mlxtend.frequent_patterns import apriori

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [4]:
import util.plot
import util.data

In [5]:
reload(util.plot)
reload(util.data);

In [6]:
with open('data/encoder.pkl', 'rb') as f:
    E = pickle.load(f)

In [7]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=1000)

In [8]:
for k in data.columns:
    if data[k].isna().sum() > 0:
        print('rm ', k)
        data.drop(columns=[k], inplace=True)

varsUsed = list(data.columns)
# varsUsed.remove('Unnamed')
varsUsed.remove('position')
varsUsed.remove('random_bool')
len(varsUsed)

rm  orig_destination_distance
rm  comp1_rate
rm  comp1_inv
rm  comp1_rate_percent_diff
rm  comp2_rate
rm  comp2_inv
rm  comp2_rate_percent_diff
rm  comp3_rate
rm  comp3_inv
rm  comp3_rate_percent_diff
rm  comp4_rate
rm  comp4_inv
rm  comp4_rate_percent_diff
rm  comp5_rate
rm  comp5_inv
rm  comp5_rate_percent_diff
rm  comp6_rate
rm  comp6_inv
rm  comp6_rate_percent_diff
rm  comp7_rate
rm  comp7_inv
rm  comp7_rate_percent_diff
rm  comp8_rate
rm  comp8_inv
rm  comp8_rate_percent_diff


48

In [9]:
E.discretizers

{'srch_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'site_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'visitor_location_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_destination_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_adults_count': KBinsDiscretizer(encode='ordinal', n_bins=array([5]), strategy='uniform'),
 'srch_children_count': KBinsDiscretizer(encode='ordinal', n_bins=array([3]), strategy='uniform'),
 'srch_room_count': KBinsDiscretizer(encode='ordinal', n_bins=array([2]), strategy='uniform'),
 'position': KBinsDiscretizer(encode='ordinal', n_bins=array([38]), strategy='uniform'),
 'srch_length_of_stay': KBinsDiscretizer(encode='ordinal',

In [16]:
# TODO encode missing data, i.e. the hotels with no clicks

# note that even though the training score is an integer,
# the predicted score is a float. The remainder terms contain information about confidence.

y_labels = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'score']
y = data['score']
x = data[varsUsed].drop(columns=y_labels)
# x.shape, y.shape
# x_train, x_test, y_train, y_test = ms.train_test_split(x, y, test_size=0.3, random_state=42)
# pd.DataFrame.drop()
xy_train, xy_test = ms.train_test_split(data, random_state=42, test_size=0.3)
y = 'score'
y_train = xy_train[y].values
y_test = xy_test[y].values
x_train = xy_train.drop(columns=[y]).values
x_test = xy_test.drop(columns=[y]).values

# y_train = y_train[:,np.newaxis]
# y_test = y_test[:,np.newaxis]

In [60]:
def cross_validation(model_func, x_train, y_train, k=None, results=None, v=0):
    # Train for 5 folds, returing ROC AUC. You can also try 'accuracy' as a scorer
    n_folds = 5
    scores_acc = cross_val_score(model_func, x_train, y_train, cv=n_folds) #  scoring='accuracy' roc_auc accuracy
#     scores_roc = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring='roc_auc') # roc_auc accuracy
    if results is not None:
        results[k] = scores_acc
    if v:
        print('scores per fold ', scores_acc)
        print('  mean score    ', np.mean(scores_acc))
        print('  standard dev. ', np.std(scores_acc))

In [79]:
models = {'GBoost 2': GradientBoostingRegressor(n_estimators=1000, learning_rate=0.001,
                                max_depth=2, random_state=0, loss='ls'),
          'GBoost 4': GradientBoostingRegressor(n_estimators=100, learning_rate=0.01,
                                max_depth=4, random_state=0, loss='ls'),          
          'RForest 2': RandomForestClassifier(n_estimators=1000, max_depth=2, random_state=0),
          'RForest 4': RandomForestClassifier(n_estimators=1000, max_depth=4, random_state=0),
          'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear', multi_class='ovr'),
          'SVR': sklearn.svm.SVR(kernel='linear'),
          'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
          'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=1000),
         }


# models = {
# #           'SGD': sklearn.linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3),
# #           'SVC auto': sklearn.svm.SVC(gamma='auto'), 
#           'SVC': sklearn.svm.SVC(kernel='linear'), 
# #           'SVC polynomial': sklearn.svm.SVC(kernel='poly', gamma='auto', degree=4),    
#           'Decision Tree':  sklearn.tree.DecisionTreeClassifier(),
#           'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
# #           'KNN 10': sklearn.neighbors.KNeighborsClassifier(n_neighbors=10),
#           'Ensemble Random Forest': sklearn.ensemble.RandomForestClassifier(n_estimators=100),
# #           'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=100)
#          }

results = {}
for k,m in models.items():
    print(k)
    cross_validation(m, x_train, y_train, k, results)
#     cross_val_score(m, x_train, y_train, cv=n_folds, v=1)

GBoost 2
GBoost 4
RForest 2
RForest 4
Logit
SVR
KNN 5
Ensemble Bagging


In [80]:
# print('Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\\\ \n\\hline')
best_k = ''
best_mean = 0
for k, scores_acc in results.items():
    if np.mean(scores_acc) > best_mean:
        best_mean = np.mean(scores_acc)
        best_k = k
    print('%s & %0.4f & %0.4f\\\\' % (k, np.mean(scores_acc), np.std(scores_acc)))
print('\nbest acc:', best_k, round(best_mean,4))

GBoost 2 & 0.8642 & 0.0009\\
GBoost 4 & 0.8655 & 0.0009\\
RForest 2 & 0.9686 & 0.0056\\
RForest 4 & 0.9886 & 0.0034\\
Logit & 0.9772 & 0.0069\\
SVR & 0.4190 & 0.2510\\
KNN 5 & 0.9615 & 0.0054\\
Ensemble Bagging & 1.0000 & 0.0000\\

best acc: Ensemble Bagging 1.0


In [81]:
for k, est in models.items():
    est.fit(x_train, y_train)
    print(k,':', mean_squared_error(y_test, est.predict(x_test)))

GBoost 2 : 0.12870701795983902
GBoost 4 : 0.1275453693298852
RForest 2 : 0.7466666666666667
RForest 4 : 0.02666666666666667
Logit : 0.10333333333333333
SVR : 0.5285970741544384
KNN 5 : 0.9866666666666667
Ensemble Bagging : 0.0
